<p style="float: left;"><a href="polymorphic-methods.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="variances.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>

# Upper Type Bounds

In Scala, [type variables](generic-classes.ipynb) may be constrained by a type bound. 

- Type bounds limit the concrete values of the type variables. 
- An _upper type bound_ or is a special case of _type constraint_, where **`T <: A` declares that `T` is subtype of `A`.**

In [ ]:
// Animals
abstract class Animal:
    def name: String

class Lion extends Animal:
  override def name: String = "Lion"

// Pets
abstract class Pet extends Animal

class Cat extends Pet:
  override def name: String = "Cat"

class Dog extends Pet:
  override def name: String = "Dog"

// Containers
class Container[A](x: A): // generic type variable, can be anything
    def elem: A = x

class AnimalContainer[A <: Animal](x: A): // notice the type constraint
    def elem: A = x

class PetContainer[A <: Pet](x: A): // notice the type constraint
    def elem: A = x

In [ ]:
// Defining some animals...
val lion = new Lion
val dog = new Dog
val cat = new Cat

Questions... 🖐️

- _Can I add the `lion` in `Container[Animal]`, why?_

In [ ]:
val container = Container[Animal](lion)

- _Can I add the `lion` in `PetContainer[Lion]`, why?_

In [ ]:
val petContainer = PetContainer[Lion](lion)

- _Can I add the `lion` and the `dog` in `AnimalContainer[Animal]`, why?_

In [ ]:
AnimalContainer[Animal](lion)

In [ ]:
AnimalContainer[Animal](dog)

- _Can I add the `dog` and the `cat` in `PetContainer[Cat]`, why?_

In [ ]:
PetContainer[Cat](dog)

In [ ]:
PetContainer[Cat](cat)

## Constraining supported types

**Some generic types and methods, even if they are designed to work for many types, only make sense when the type parameter supports certain behavior/s.**

For example, for a type `Set[A]` to work correctly, 

- the elements of type `A` must be comparable.
- We capture this requirement by stating that `A` is a subtype of `Comparable[A]`.

In [ ]:
import scala.language.postfixOps

trait Comparable[A]:
    def compare(that: A): Int
    def < (that: A): Boolean = (this `compare` that) < 0
    def > (that: A): Boolean = (this `compare` that) > 0
    def <= (that: A): Boolean = (this `compare` that) <= 0
    def >= (that: A): Boolean = (this `compare` that) >= 0
    def compareTo(that: A): Int = compare(that)

Every element type `A` in `Set[A]` must be a subtype of `Comparable[A]`.

We enforce this requirement by adding the type constraint `[A <: Comparable[A]]` in the definition of `Set`.

In [ ]:
import scala.language.postfixOps

abstract class Set[A <: Comparable[A]]:
    def incl(x: A): Set[A]
    def contains(x: A): Boolean

class EmptySet[A <: Comparable[A]] extends Set[A]:
    def contains(x: A): Boolean = false
    def incl(x: A): Set[A] = new NonEmptySet(x, new EmptySet[A], new EmptySet[A])

class NonEmptySet[A <: Comparable[A]](e: A, lset: Set[A], rset: Set[A]) extends Set[A]:
    def contains(x: A): Boolean =
        if (x < e) lset `contains` x // T needs to be comparable
        else if (x > e) rset `contains` x
        else true
    def incl(x: A): Set[A] =
        if (x < e) new NonEmptySet(e, lset `incl` x, rset)
        else if (x > e) new NonEmptySet(e, lset, rset `incl` x)
        else this

Finally, we define the type `Num` which is subtype of `Comprable[Num]`.

In [ ]:
case class Num(value: Double) extends Comparable[Num] {
    def compare(that: Num): Int =
        if (this.value < that.value) -1
        else if (this.value > that.value) 1
        else 0
}

val s = new EmptySet[Num].incl(Num(9)).incl(Num(0)).incl(Num(10))

s.contains(Num(0)) // true
s.contains(Num(1001)) // false


<span style="color:red">**We are required only to implement the `compare` method, because the other methods and operators use the definition of it.**</span>

<p style="float: left;"><a href="polymorphic-methods.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="variances.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>